# Combined data
This notebook aims at combining the different datasets that was generated with the houses dataset

df_with_latlong.csv: main dataset

mrt_lrt_data.csv: dataset containing coordinates of all MRT and LRT stations 

schools_data.csv: dataset containing coordinates of all primary and secondary schools

malls_data.csv: dataset containing coordinates of all shopping malls

hawkers_data.csv: dataset containing coordinates of all hawker centres

counts_df.csv: dataset containing coordinates of all unique property coordinates and the corresponding travel time to Raffles Place MRT

In [2]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

import re

### Combine MRT data with houses data

This section will calculate the distance to the nearest MRT and record down the station name of that MRT station. This section uses the geopy library.

In [7]:
df = pd.read_csv('df_with_latlong.csv')
mrt_df = pd.read_csv('mrt_lrt_data.csv')

In [6]:
mrt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_name  157 non-null    object 
 1   type          157 non-null    object 
 2   lat           157 non-null    float64
 3   lng           157 non-null    float64
dtypes: float64(2), object(2)
memory usage: 5.0+ KB


#### Code below will take some time to run (about 2h)

In [8]:
from geopy.distance import geodesic

# Define a new function to be applied for faster computation
def find_nearest_mrt(row):
    min_distance = float('inf')
    nearest_mrt = ''
    
    for idx, mrt in mrt_df.iterrows():
        distance = geodesic((row['Latitude'], row['Longitude']), (mrt['lat'], mrt['lng'])).km
        
        if distance < min_distance:
            min_distance = distance
            nearest_mrt = mrt['station_name']
    
    return pd.Series({'nearest MRT': nearest_mrt, 'distance to nearest MRT': min_distance})

df[['nearest MRT', 'distance to nearest MRT']] = df.apply(find_nearest_mrt, axis=1)


In [9]:
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Longitude,Latitude,Address,Postal code,nearest MRT,distance to nearest MRT
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61,232000.0,103.853880,1.362005,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,560406,Ang Mo Kio,1.007304
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60,250000.0,103.838202,1.370966,108 ANG MO KIO AVENUE 4 KEBUN BARU HEIGHTS SIN...,560108,Ang Mo Kio,1.271398
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62,262000.0,103.835368,1.380709,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,560602,Yio Chu Kang,1.069720
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62,265000.0,103.857201,1.366201,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,560465,Ang Mo Kio,0.946884
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62,265000.0,103.835132,1.381041,601 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,560601,Yio Chu Kang,1.092577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148272,2023-03,YISHUN,5 ROOM,386,YISHUN RING RD,10 TO 12,127.0,Improved,1988,64,629000.0,103.846473,1.428981,386 YISHUN RING ROAD SINGAPORE 760386,760386,Yishun,1.274187
148273,2023-03,YISHUN,5 ROOM,221,YISHUN ST 21,01 TO 03,126.0,Improved,1985,61,565000.0,103.836068,1.433722,221 YISHUN STREET 21 YISHUN GARDENS SINGAPORE ...,760221,Yishun,0.462724
148274,2023-03,YISHUN,5 ROOM,335C,YISHUN ST 31,13 TO 15,112.0,Improved,2015,91,625000.0,103.844367,1.432488,335C YISHUN STREET 31 YISHUN RIVERWALK SINGAPO...,763335,Yishun,1.083451
148275,2023-03,YISHUN,5 ROOM,820,YISHUN ST 81,07 TO 09,122.0,Improved,1988,64,650000.0,103.833868,1.413099,820 YISHUN STREET 81 SINGAPORE 760820,760820,Khatib,0.487889


In [10]:
df.to_csv('df_with_latlong.csv',index=False) # Checkpoint

### Combine schools data with houses data

This section will calculate the number of schools within a 1km range of the property. This section uses the geopy library.

In [12]:
df_schools = pd.read_csv('schools_data.csv')

In [17]:
%%time
from geopy.distance import geodesic

# Create a list of tuples containing the school coordinates
school_coords = [(row['Latitude'], row['Longitude']) for index, row in df_schools.iterrows()]

# Function to find number of schools within a certain radius
def count_schools_within_radius(lat, long, radius):
    distances = [geodesic((lat, long), coord).km for coord in school_coords]
    schools_within_radius = sum(distance <= radius for distance in distances)
    return schools_within_radius

# Apply the function to create the new column
df['schools within 1km'] = [count_schools_within_radius(row['Latitude'], row['Longitude'], 1) for index, row in df.iterrows()]

CPU times: user 2h 32min 41s, sys: 15.9 s, total: 2h 32min 57s
Wall time: 2h 33min 7s


In [21]:
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Longitude,Latitude,Address,Postal code,nearest MRT,distance to nearest MRT,schools within 1km
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61,232000.0,103.853880,1.362005,406 ANG MO KIO AVENUE 10 SINGAPORE 560406,560406,Ang Mo Kio,1.007304,3
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60,250000.0,103.838202,1.370966,108 ANG MO KIO AVENUE 4 KEBUN BARU HEIGHTS SIN...,560108,Ang Mo Kio,1.271398,7
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62,262000.0,103.835368,1.380709,602 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,560602,Yio Chu Kang,1.069720,6
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62,265000.0,103.857201,1.366201,465 ANG MO KIO AVENUE 10 TECK GHEE HORIZON SIN...,560465,Ang Mo Kio,0.946884,3
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62,265000.0,103.835132,1.381041,601 ANG MO KIO AVENUE 5 YIO CHU KANG GREEN SIN...,560601,Yio Chu Kang,1.092577,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148272,2023-03,YISHUN,5 ROOM,386,YISHUN RING RD,10 TO 12,127.0,Improved,1988,64,629000.0,103.846473,1.428981,386 YISHUN RING ROAD SINGAPORE 760386,760386,Yishun,1.274187,3
148273,2023-03,YISHUN,5 ROOM,221,YISHUN ST 21,01 TO 03,126.0,Improved,1985,61,565000.0,103.836068,1.433722,221 YISHUN STREET 21 YISHUN GARDENS SINGAPORE ...,760221,Yishun,0.462724,9
148274,2023-03,YISHUN,5 ROOM,335C,YISHUN ST 31,13 TO 15,112.0,Improved,2015,91,625000.0,103.844367,1.432488,335C YISHUN STREET 31 YISHUN RIVERWALK SINGAPO...,763335,Yishun,1.083451,7
148275,2023-03,YISHUN,5 ROOM,820,YISHUN ST 81,07 TO 09,122.0,Improved,1988,64,650000.0,103.833868,1.413099,820 YISHUN STREET 81 SINGAPORE 760820,760820,Khatib,0.487889,5


In [20]:
df.to_csv('df_with_latlong.csv',index=False) # Checkpoint

### Combine malls data with houses data

This section will calculate the distance to the nearest mall and record down the name of that mall. It will also calculate the number of malls within a 500m and 1km radius. This section uses the geopy library.

In [22]:
malls_df = pd.read_csv('malls_data.csv')

In [24]:
%%time
from geopy.distance import geodesic

# Function to find nearest mall and distance to nearest mall
def find_nearest_mall(lat, long):
    distances = malls_df.apply(lambda row: geodesic((lat, long), (row['Latitude'], row['Longitude'])).km, axis=1)
    nearest_mall_distance = min(distances)
    nearest_mall_index = distances.idxmin()
    nearest_mall = malls_df.loc[nearest_mall_index, 'Mall']
    return pd.Series({'nearest mall': nearest_mall, 'mall nearest distance': nearest_mall_distance})

# Function to find number of malls within a certain radius
def count_malls_within_radius(lat, long, radius):
    distances = malls_df.apply(lambda row: geodesic((lat, long), (row['Latitude'], row['Longitude'])).km, axis=1)
    malls_within_radius = (distances <= radius).sum()
    return malls_within_radius

# Apply the functions to create the new columns
df[['nearest mall', 'mall nearest distance']] = df.apply(lambda row: find_nearest_mall(row['Latitude'], row['Longitude']), axis=1)
df['mall within 500m'] = df.apply(lambda row: count_malls_within_radius(row['Latitude'], row['Longitude'], 0.5), axis=1)
df['mall within 1km'] = df.apply(lambda row: count_malls_within_radius(row['Latitude'], row['Longitude'], 1), axis=1)


CPU times: user 3h 5min 32s, sys: 22.7 s, total: 3h 5min 55s
Wall time: 3h 6min 19s


In [25]:
df_hawkers = pd.read_csv('hawkers_data.csv')
df_hawkers

,Name,Latitude,Longitude
0,Market Street Hawker Centre,1.284425,103.850165
1,Marsiling Mall Hawker Centre,1.433539,103.779785
2,Margaret Drive Hawker Centre,1.297486,103.804715
3,Fernvale Hawker Centre & Market,1.391592,103.877060
4,One Punggol Hawker Centre,1.408190,103.904806
...,...,...,...
120,West Coast Drive Blk 503 (Ayer Rajah Food Centre),1.311803,103.759751
121,Whampoa Drive Blk 90 (Whampoa Drive Makan Plac...,1.322943,103.855138
122,Whampoa Drive Blk 91/92 (Whampoa Drive Makan P...,1.323422,103.854068
123,Yishun Park Hawker Centre,1.425018,103.844748


### Combine hawkers data with houses data
This section will calculate the number of hawker centres within a 1km range of the property. This section uses the geopy library.

In [26]:
%%time
from geopy.distance import geodesic

# Function to find number of hawkers within a certain radius
def count_hawkers_within_radius(lat, long, radius):
    distances = df_hawkers.apply(lambda row: geodesic((lat, long), (row['Latitude'], row['Longitude'])).km, axis=1)
    return (distances <= radius).sum()

# Apply the function to create the new column
df['hawkers within 1km'] = df.apply(lambda row: count_hawkers_within_radius(row['Latitude'], row['Longitude'], 1), axis=1)


CPU times: user 39min 36s, sys: 3.06 s, total: 39min 39s
Wall time: 39min 39s


### Export all common LatLong data
To find out how long it takes to get from the stated address to Raffles Place MRT (CBD).

Exporting this out and processing it externally as I am using Google's API and my bank account is linked to it, and I still need the money to pay my NTU school feels. Please see the code snippet under 1e. Extract Travel Time to RP data. Hopefully I haven't accidentally revealed my key.

In [74]:
df['travel time to RP in minutes'] = np.nan

In [76]:
# Combining LatLong data as string and put the unique items into a dataframe
counts_df = pd.concat([df['Latitude'].to_frame(), df['Longitude'].to_frame()], axis=1)
counts_df['LatLong'] = counts_df['Latitude'].astype(str) + ',' + counts_df['Longitude'].astype(str)
counts_df = counts_df['LatLong'].value_counts().to_frame().reset_index()
counts_df.columns = ['LatLong', 'Count']
counts_df

,LatLong,Count
0,"1.40776143065745,103.900998963843",138
1,"1.2957304268451,103.809759937443",112
2,"1.40800694944027,103.899074357998",109
3,"1.33627775831234,103.844044881122",108
4,"1.29579395038247,103.809264765809",101
...,...,...
10305,"1.35641014489862,103.945141459551",1
10306,"1.35832151751736,103.750135909014",1
10307,"1.36177996910038,103.750528224559",1
10308,"1.35546395095099,103.935138528274",1


In [77]:
# We can see that the number of houses still remains the same
counts_df['Count'].sum()

148277

In [78]:
counts_df.to_csv('counts_df.csv', index=False)

#### Back from processing travel time data

In [11]:
counts_df = pd.read_csv('counts_df.csv')
counts_df

,LatLong,nearest MRT,Count,travel time to RP in minutes
0,"1.36200453938712,103.853879910407",Ang Mo Kio,13,40
1,"1.37096635222625,103.838201940326",Ang Mo Kio,18,44
2,"1.38070883044887,103.835368226602",Yio Chu Kang,19,50
3,"1.3662010408294,103.857200967235",Ang Mo Kio,28,43
4,"1.38104135523576,103.835131742647",Yio Chu Kang,22,47
...,...,...,...,...
10305,"1.41240459366921,103.900177817489",Nibong,3,57
10306,"1.44576016585328,103.814267716736",Sembawang,1,55
10307,"1.36394578780556,103.954621464542",Tampines East,1,55
10308,"1.42145242328288,103.84332790779",Khatib,4,54


In [15]:
# Decided that having the latlong data together is much better for processing
df['LatLong'] = df['Latitude'].astype(str) + ',' + df['Longitude'].astype(str)
df = df.drop(columns=['Latitude', 'Longitude'])

In [90]:
df.to_csv('df_with_latlong.csv',index=False) # Checkpoint

In [13]:
counts_df = counts_df.drop(columns=['Count', 'nearest MRT'])

# Left merge dataframe to ensure that common values are merged
df = pd.merge(df, counts_df, on='LatLong', how='left')
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,...,nearest MRT,distance to nearest MRT,schools within 1km,nearest mall,mall nearest distance,mall within 500m,mall within 1km,hawkers within 1km,LatLong,travel time to RP in minutes
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61,...,Ang Mo Kio,1.007304,3,AMK Hub,0.997504,0,1,3,"1.36200453938712,103.853879910407",40
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60,...,Ang Mo Kio,1.271398,7,Broadway Plaza,0.868861,0,1,4,"1.37096635222625,103.838201940326",44
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62,...,Yio Chu Kang,1.069720,6,Broadway Plaza,1.525573,0,0,2,"1.38070883044887,103.835368226602",50
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62,...,Ang Mo Kio,0.946884,3,myVillage At Serangoon Garden,0.893796,0,1,3,"1.3662010408294,103.857200967235",43
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62,...,Yio Chu Kang,1.092577,5,Broadway Plaza,1.569307,0,0,2,"1.38104135523576,103.835131742647",47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148268,2023-03,YISHUN,5 ROOM,386,YISHUN RING RD,10 TO 12,127.0,Improved,1988,64,...,Yishun,1.274187,3,Junction Nine,0.701213,0,1,1,"1.42898064926516,103.846473489722",50
148269,2023-03,YISHUN,5 ROOM,221,YISHUN ST 21,01 TO 03,126.0,Improved,1985,61,...,Yishun,0.462724,9,Northpoint City,0.620131,0,2,1,"1.43372181338636,103.836068035067",48
148270,2023-03,YISHUN,5 ROOM,335C,YISHUN ST 31,13 TO 15,112.0,Improved,2015,91,...,Yishun,1.083451,7,Junction Nine,0.309025,1,1,1,"1.43248777014111,103.844367368405",55
148271,2023-03,YISHUN,5 ROOM,820,YISHUN ST 81,07 TO 09,122.0,Improved,1988,64,...,Khatib,0.487889,5,Wisteria Mall,0.983081,0,1,0,"1.41309901104205,103.833867702757",47


In [100]:
df.to_csv('df_with_latlong.csv',index=False) # Checkpoint

# Final dataset

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148273 entries, 0 to 148272
Data columns (total 23 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   month                         148273 non-null  object 
 1   town                          148273 non-null  object 
 2   flat_type                     148273 non-null  object 
 3   block                         148273 non-null  object 
 4   street_name                   148273 non-null  object 
 5   storey_range                  148273 non-null  object 
 6   floor_area_sqm                148273 non-null  float64
 7   flat_model                    148273 non-null  object 
 8   lease_commence_date           148273 non-null  int64  
 9   remaining_lease               148273 non-null  int64  
 10  resale_price                  148273 non-null  float64
 11  Address                       148273 non-null  object 
 12  Postal code                   148273 non-nul